# Checking enviroment states and action sets

In [1]:
import sys
sys.path.append("..")
from SARSA_Agent import *
from ENV_DETAILS import *
from RUN_TENSORBOARD import *

events_folder = "./logs_hyper"
main("./logs_hyper")

2024-03-11 16:00:32.770622: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Selected port: 47029


2024-03-11 16:00:34.510996: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:47029/ (Press CTRL+C to quit)


In [2]:

ENV = "LunarLander-v2"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:
env = gym.make(ENV)#,new_step_api=True
env

<TimeLimit<OrderEnforcing<LunarLander<LunarLander-v2>>>>

In [4]:
s = env.observation_space.sample()
s, s.shape

(array([ 1.12129   , -0.8256469 ,  1.9506097 ,  1.41141   , -1.356025  ,
        -0.08034442, -0.86012703, -0.81464326], dtype=float32),
 (8,))

In [5]:
env.observation_space, env.observation_space.shape

(Box(-inf, inf, (8,), float32), (8,))

In [6]:
s = env.reset()
env.action_space.n, env.action_space, env.step(1)

(4,
 Discrete(4),
 (array([ 0.00774088,  1.4172187 ,  0.3865181 ,  0.12703912, -0.00727374,
         -0.05514999,  0.        ,  0.        ], dtype=float32),
  1.110616457898858,
  False,
  {}))

### Training/Hyperparam run

In [7]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_SARSA/"

In [8]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_SARSA"
    exploration_tech = "epsilon"

    project_name = "keras_tunning_"+exploration_tech
    tuner = kt.BayesianOptimization(
        MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/SARSA_"+exploration_tech+"/", exploration_tech =exploration_tech,
                        sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value = SUCESS_CRITERIA_VALUE,
                end_of_episode = EPISODES, batch = 32,  evaluation_epoch = 2000, 
                training_steps = 700000, 
                time_to_update_min = 200, time_to_update_max = 800,
                lr_min = 0.000005, lr_max = 0.005,
                discount_min = 0.90, discount_max = 0.99,
                dense_min = 32, dense_max = 256,
                environment_name = ENV),
        objective= kt.Objective('total_train_reward', direction="max"), 
        max_trials = 50,
        # distribution_strategy=tf.distribute.MirroredStrategy(),
        directory= dir,
        project_name=project_name
    )
    tuner.search(x=[0], y=[1])

    exploration_tech = "boltzman"

    project_name = "keras_tunning_"+exploration_tech
    tuner = kt.BayesianOptimization(
        MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/SARSA_"+exploration_tech+"/", exploration_tech =exploration_tech,
                        sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value = SUCESS_CRITERIA_VALUE,
                end_of_episode = EPISODES, batch = 32,  evaluation_epoch = 2000, 
                training_steps = 700000, 
                time_to_update_min = 200, time_to_update_max = 800,
                lr_min = 0.000005, lr_max = 0.005,
                discount_min = 0.90, discount_max = 0.99,
                dense_min = 32, dense_max = 256,
                environment_name = ENV),
        objective= kt.Objective('total_train_reward', direction="max"), 
        max_trials = 50,
        # distribution_strategy=tf.distribute.MirroredStrategy(),
        directory= dir,
        project_name=project_name
    )
    tuner.search(x=[0], y=[1])


else : 
    print("Acquiring parameters ....")
    learning_rate = 0.1
    training_steps = 5000000
    exploration_tech = "epsilon"
    discount = 0.99
    time_to_update = 100
    max_n_layers = 1
    dense_units = [32]
    end_of_episode = EPISODES
    sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH
    sucess_criteria_value = SUCESS_CRITERIA_VALUE
    run_training(training_steps, learning_rate,  exploration_tech, discount, time_to_update, max_n_layers, dense_units, writer, end_of_episode, sucess_criteria_epochs, sucess_criteria_value)

Reloading Tuner from Hyperparam_kt_SARSA/keras_tunning_epsilon/tuner0.json


In [9]:
val_env = gym.make(ENV)

for exploration_tech in ["epsilon", "boltzman"]:

    project_name = "keras_tunning_"+exploration_tech
    tuner = kt.BayesianOptimization(
        MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/SARSA_"+exploration_tech+"/", exploration_tech =exploration_tech,
                        sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value = SUCESS_CRITERIA_VALUE,
                end_of_episode = EPISODES, batch = 32,  evaluation_epoch = 2000, 
                training_steps = 700000, 
                time_to_update_min = 200, time_to_update_max = 800,
                lr_min = 0.000005, lr_max = 0.005,
                discount_min = 0.90, discount_max = 0.99,
                dense_min = 32, dense_max = 256,
                environment_name = ENV),
        objective= kt.Objective('total_train_reward', direction="max"), 
        max_trials = 50,
        directory= dir,
        project_name=project_name
    )

    for trials in tuner.oracle.get_best_trials(num_trials=3):
        print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)
        

    print(tuner.get_best_models())
    env_model = tuner.get_best_models()[0]
    final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./SARSA_"+exploration_tech+"_video.mp4")
    print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))

    

Reloading Tuner from Hyperparam_kt_SARSA/keras_tunning_boltzman/tuner0.json
Trial id :24 | Score :-13.884199567491873 --> {'discount': 0.9500000000000001, 'learning_rate': 0.0017373768696360733, 'time_to_update': 400, 'n_dense_layers': 2, 'dense_units_0': 196, 'dense_units_1': 106}
Trial id :22 | Score :-19.616048433071466 --> {'discount': 0.9400000000000001, 'learning_rate': 0.0014975550737667965, 'time_to_update': 400, 'n_dense_layers': 2, 'dense_units_0': 183, 'dense_units_1': 118}
Trial id :43 | Score :-23.511146140800573 --> {'discount': 0.9, 'learning_rate': 0.00015485484197871015, 'time_to_update': 200, 'n_dense_layers': 2, 'dense_units_0': 117, 'dense_units_1': 122}
Trial number :  50
Trial number :  50
0.0059099197


ValueError: Exception encountered when calling layer "sarsa_agent" "                 f"(type SARSAAgent).

In this `tf.Variable` creation, the initial value's shape ((106, 4)) is not compatible with the explicitly supplied `shape` argument ((32, 4)).

Call arguments received by layer "sarsa_agent" "                 f"(type SARSAAgent):
  • observations=tf.Tensor(shape=(1, 1), dtype=float32)